# Spark Streaming Four Square Challenge - Stream Reading Notebook
Raj Prasad
July 2019

[html version](https://daddyprasad5.github.io/foursquare_streaming_challenge.html) 

[jupyter notebook version](https://github.com/daddyprasad5/thinkful/blob/master/foursquare_streaming_challenge.ipynb) 

The goal of this exercise is to create a stream of foursquare trending venues by calling the foursquare APIs and storing the output into files in my google drive.  There's [another notebook](https://github.com/daddyprasad5/thinkful/blob/master/foursquare_client.ipynb) that does this part.  

This notebook then reads that stream and displays data from it - nothing fancy just the print of the dataframe.  

I've got [another interesting version](https://daddyprasad5.github.io/interactive_gmaps_and_four_square.html) of this that uses Jupyter Widgets to allow the notebook user to pick a city, then collects the foursquare trending venues from the foursquare API, and displays the locations of those venues on a google map.   

In [3]:
#install spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz

# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark
!pip install gmaps
!pip install ipywidgets
!pip install widgetsnbextension

# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

# Point Colaboratory to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#imports
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.functions import desc, col, window

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from pyspark.streaming import StreamingContext

from IPython.display import display,HTML,clear_output

import json
import time
import os
import requests
import datetime
import pandas as pd

In [0]:
#set constants
APP_NAME = "Foursquare streaming app"
SPARK_URL = "local[*]"
RANDOM_SEED = 141107
TRAINING_DATA_RATIO = 0.8
LOG_PATH = "/content/gdrive/My Drive/Colab Datasets/foursquare_logs/"

In [6]:
inputPath = LOG_PATH

numFiles = len(os.listdir(inputPath))
numFileOffset = numFiles - 1

print(f"There are {numFiles} files in our inputPath, which gives an offset of {numFileOffset}.")

spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

There are 12 files in our inputPath, which gives an offset of 11.


In [0]:
# Static DataFrame representing data in the JSON files
staticInputDF = spark.read.json(inputPath)

In [18]:
staticInputDF.printSchema()

root
 |-- city: string (nullable = true)
 |-- locations: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [20]:
flowSchema = staticInputDF.schema

StructType(List(StructField(city,StringType,true),StructField(locations,ArrayType(ArrayType(DoubleType,true),true),true),StructField(names,ArrayType(StringType,true),true)))

In [0]:
streamingInputDF = (
  spark
    .readStream                       
    .schema(flowSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

In [0]:
query.stop()

In [34]:
streamingLocationsDF = streamingInputDF \
    .select('city', "locations", "names") \
    .groupby('city', "locations", "names") \
    .count()

# Is this DF actually a streaming DF?
streamingLocationsDF.isStreaming

True

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingLocationsDF
    .writeStream
    .format("memory")       
    .queryName("trending_locations")   
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

In [37]:
while True:
    streamingLocationsDF = spark.sql("select * from trending_locations").collect()
    display(streamingLocationsDF)
    time.sleep(10)

[Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=2),
 Row(city='San Francisco', locations=[], names=[], count=2),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.351490065921098]], names=['Paris'], count=1)]

[Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=2),
 Row(city='San Francisco', locations=[], names=[], count=2),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.351490065921098]], names=['Paris'], count=2)]

[Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=2),
 Row(city='San Francisco', locations=[], names=[], count=2),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.351490065921098]], names=['Paris'], count=2)]

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=1),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=2),
 Row(city='San Francisco', locations=[], names=[], count=2),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=1),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=3),
 Row(city='San Francisco', locations=[], names=[], count=2),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

...

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=2),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=3),
 Row(city='San Francisco', locations=[], names=[], count=3),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=2),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=3),
 Row(city='San Francisco', locations=[], names=[], count=3),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=2),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=3),
 Row(city='San Francisco', locations=[], names=[], count=3),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

[Row(city='London', locations=[[51.51464, -0.152864], [51.516185764675825, -0.14046166652284045], [51.51429953268523, -0.15159845352172852]], names=['Selfridges & Co', 'JOLT', 'EL&N'], count=2),
 Row(city='New York City', locations=[[40.753621, -73.983265], [40.750497022729604, -73.99327286150431], [40.74226204193276, -73.98800611495972], [40.73883630073037, -73.99277694087336], [40.74610215801135, -73.98206990777314], [40.75267443042928, -73.97697960643174], [40.73491348804492, -74.00205073261284]], names=['Bryant Park', 'New York Penn Station', 'Madison Square Park', 'OOTOYA 大戸屋 (OOTOYA)', 'Pret A Manger', 'Grand Central Terminal', 'Partners Coffee Roasters'], count=3),
 Row(city='San Francisco', locations=[], names=[], count=3),
 Row(city='London', locations=[[51.51464, -0.152864], [51.51429953268523, -0.15159845352172852], [51.516185764675825, -0.14046166652284045]], names=['Selfridges & Co', 'EL&N', 'JOLT'], count=1),
 Row(city='Paris', locations=[[48.857005585002405, 2.3514900659

KeyboardInterrupt: ignored

In [33]:
for r in streamingLocationsDF.collect():
  for v in (r.response.venues):
    print(v.name)
    print((v.location.lat, v.location.lng))

Selfridges & Co
(51.51464, -0.152864)
JOLT
(51.516185764675825, -0.14046166652284045)
EL&N
(51.51429953268523, -0.15159845352172852)
Piccadilly Circus
(51.50996110016709, -0.1346522569656372)
Mayfair
(51.51135999349115, -0.1472640037536621)
Ferdi
(51.50650181288355, -0.14646190547074883)
EL&N
(51.506815937331424, -0.15188813209533691)
Selfridges & Co
(51.51464, -0.152864)
JOLT
(51.516185764675825, -0.14046166652284045)
EL&N
(51.51429953268523, -0.15159845352172852)
Piccadilly Circus
(51.50996110016709, -0.1346522569656372)
London
(51.50767403407925, -0.12788772583007812)
Ferdi
(51.50650181288355, -0.14646190547074883)
New York Penn Station
(40.75032192413809, -73.99280227613797)
Bryant Park
(40.753621, -73.983265)
Grand Central Terminal
(40.75267443042928, -73.97697960643174)
Go! Go! Curry!
(40.741295408672435, -73.99658313107955)
Bryant Park
(40.753621, -73.983265)
Washington Square Park
(40.73081579656163, -73.9974582195282)
Go! Go! Curry!
(40.741295408672435, -73.99658313107955)
Mad

In [0]:
query.stop()

In [0]:
staticInputDF